In [20]:
#Use 3.12.4
import openai
import os
import copy
import time
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.docstore.document import Document

key_path = '/Users/jaesolshin/key/openai_key.txt'
os.environ["OPENAI_API_KEY"] = open(key_path, 'r', encoding='utf-8').read()
openai.api_key = os.environ["OPENAI_API_KEY"] 

In [21]:
#!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

import fitz  # PyMuPDF
from pykospacing import Spacing

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_path = '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf'
extracted_text = extract_text_from_pdf(pdf_path)

spacing = Spacing()
corrected_text = spacing(extracted_text)

In [22]:
corrected_documents = Document(page_content=corrected_text)

In [23]:
from langchain.text_splitter import SpacyTextSplitter
from langchain.docstore.document import Document

corrected_documents = [Document(page_content=corrected_text)]

text_splitter = SpacyTextSplitter(
    chunk_size=2000, #분할할 크기를 설정
    chunk_overlap=10,
    pipeline='ko_core_news_sm' #분할에 사용할 언어모델을 설정
)

splitted_documents = text_splitter.split_documents(corrected_documents) #문서를 분할

print(f'분할 전 문서 개수: {len(corrected_documents)}')
print(f'분할 후 문서 개수: {len(splitted_documents)}')

Created a chunk of size 3367, which is longer than the specified 2000
Created a chunk of size 2707, which is longer than the specified 2000


분할 전 문서 개수: 1
분할 후 문서 개수: 113


In [24]:
for doc in splitted_documents:
    print(doc)

page_content='ISSN 1975-4922 2023 연차보고서 2024.

3 ｢ 한국은행법 ｣ 제102조(연차보고서의 공표) ① 한국은행은 매 회계연도가 지난 후 3개월 이내에 해 당 회계연도 중의 한국은행 의 업무 상태와 통화 및 정부의 외환에 관한 정책을 개략적으로 기술하고 금융경제 상태를 분석한 연차보고서를 정부에 제출하고, 이를 공표하여야 한다.

② 제1항에 따른  연차보고서는 금융통화위원회의 의결을 거쳐야 한다.

총재 서문 한국은행은 우리나라 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가안정과 금융안정을 도모함으로써 국민경제의 건전한 발전에 이바지하고 자 최선을 다하고 있습니다.

2023년은 급변하는 안팎의 여건에 대응하여 보다 나은 정책과 해법을 마련 하는데 분주했던 한 해였습니다.

인플레이션을 잡기 위한 주요국 중앙은행들 의 가파른 금리 인상에 이어 미국 실리콘 밸리은행 사태, 이스라엘-하마스 전쟁 등으로 경계의 끈을 한시도 늦출 수 없는 긴장된 순간의 연속이었습니다.

이 처럼 높아진 대외여건의 불확실성 속에서 한국은행은 최우선으로 물가안정을 도모하는 가운데 금융안정 측면의 리스크를 면밀히 점검하며 통화정책을 운영 하였습니다.

기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금 인출 사태 등 금융･외환시장 불안에는 시장안정화 조치를 통해 적극 대처하 였습니다.

한국은  행은 경제상황에 대한 정확한 진단과정교한 정책 대응을 뒷받침하고자 조사 및 정책연구 업무를 적극적으로 수행하였습니다.

초저출산 및 초고령사회 의 극단적인 구구조 문제, 장기 구조적 관점에서 진단한 가계부채 현황, 지역 간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시했습니다.

또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다 각적으로 분석하는 한편 지급결제 환경 변화에도 선제적으로 대응하기 위해 중앙은행 디지털 화폐(CBDC) 활용성 테스트를 관계기관과 함께 추진하고 관련 연구를 이어나갔

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 모델 정의
chat = ChatOpenAI(
    model='gpt-4o-mini-2024-07-18',
)


# 2단계 프롬프트(표, 목차 제거)
examples2 = [
    {
        'input':"Ⅱ- 1. 주요 이슈 44 Ⅱ- 2.",
        'output':"\n"
    },
    {
        'input':"표 Ⅱ- 13. 외화자산 구성 내역 (기말 기준) (%, %p) 2021 2022(A) 2023(B) 증감(B-A) ",
        'output':"\n"
    },
    {
        'input':"127) 한국은행의 업무 관련 지식, 정책 수행 경험...",
        'output':"\n"
    },
    {
        'input':"주요 품목별 수출 (%)\n 2022 2023\n 증감률 비중 증감률 비중\n IT제품 2.5 <34.1> -19.9 <29.5> ...",
        'output':"\n"
    },
]

prompt = PromptTemplate(
    input_variables=['input', 'output'],
    template='입력: {input}\n출력: {output}'
)


few_shot_prompt = FewShotPromptTemplate(
    examples=examples2,
    example_prompt=prompt,
    prefix="예시와 같이 표나 차례, 주석 형식의 불필요한 텍스트를 제거하세요. 1) 한국어 문법에 맞는 문장이 아닌 기호나 숫자, 단어 등이 나열된 목차 형식의 텍스트 제거한다. 2) 기호나 숫자, 단어 등이 나열된 표 형식의 데이터 제거한다. 3) 주석은 '127)'과 같이 숫자로 시작하는 문장은 주석이므로 제거한다. 4)  문체는 '입니다'가 아닌 '이다'체로 통일한다.",
    suffix="입력: {input_string}\n출력:",
    input_variables=['input_string'],
)

rectified_documents_step2 =[]

def rectify2(phrase):
    formatted_prompt = few_shot_prompt.format(
        input_string=phrase
    )
    result = chat(
        [
            SystemMessage(content='당신은 신문사에서 전문적인 교열을 담당하는 편집자입니다. 요구사항에 맞게 주어진 텍스트를 수정하세요.'),
            HumanMessage(content=formatted_prompt)
        ]
    )
    
    return result.content


for idx, doc in enumerate(splitted_documents):
    print(f'processing {idx+1}')
    rect = rectify2(doc.page_content)
    
    # '입력:'이나 '출력:'이 포함되어 있는지 확인
    if '입력:' not in rect and '출력:' not in rect:
        rectified_doc = Document(page_content=rect, metadata=doc.metadata)
        rectified_documents_step2.append(rectified_doc)
    else:
        print(f'Skipping document {idx+1} due to "입력:" or "출력:" being present.')
    
    time.sleep(1)


processing 1
processing 2
processing 3
processing 4
processing 5
processing 6
processing 7
processing 8
processing 9
processing 10
processing 11
Skipping document 11 due to "입력:" or "출력:" being present.
processing 12
Skipping document 12 due to "입력:" or "출력:" being present.
processing 13
processing 14
processing 15
processing 16
processing 17
Skipping document 17 due to "입력:" or "출력:" being present.
processing 18
processing 19
processing 20
processing 21
processing 22
processing 23
processing 24
processing 25
processing 26
processing 27
processing 28
processing 29
processing 30
processing 31
processing 32
processing 33
processing 34
processing 35
processing 36
processing 37
processing 38
processing 39
processing 40
processing 41
processing 42
processing 43
Skipping document 43 due to "입력:" or "출력:" being present.
processing 44
processing 45
processing 46
processing 47
processing 48
processing 49
Skipping document 49 due to "입력:" or "출력:" being present.
processing 50
processing 51
proce

In [26]:
for doc in rectified_documents_step2:
    print(doc.page_content)

주요 이슈

총재 서문 한국은행은 우리나라 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가안정과 금융안정을 도모함으로써 국민경제의 건전한 발전에 이바지하고 최선을 다하고 있다.

2023년은 급변하는 안팎의 여건에 대응하여 보다 나은 정책과 해법을 마련하는 데 분주했던 한 해였다.

인플레이션을 잡기 위한 주요국 중앙은행들의 가파른 금리 인상에 이어 미국 실리콘 밸리은행 사태, 이스라엘-하마스 전쟁 등으로 경계의 끈을 한시도 늦출 수 없는 긴장된 순간의 연속이었다.

이처럼 높아진 대외여건의 불확실성 속에서 한국은행은 최우선으로 물가안정을 도모하는 가운데 금융안정 측면의 리스크를 면밀히 점검하며 통화정책을 운영하였다.

기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금 인출 사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다.

한국은행은 경제상황에 대한 정확한 진단과 교묘한 정책 대응을 뒷받침하고자 조사 및 정책연구 업무를 적극적으로 수행하였다.

초저출산 및 초고령사회의 극단적인 구조 문제, 장기 구조적 관점에서 진단한 가계부채 현황, 지역 간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다.

또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각적으로 분석하는 한편 지급결제 환경 변화에도 선제적으로 대응하기 위해 중앙은행 디지털 화폐(CBDC) 활용성 테스트를 관계기관과 함께 추진하고 관련 연구를 이어나갔다.

한국은행은 국제사회에서 우리나라의 높아진 위상에 부합하는 역할을 수행하기 위해 국제기구, 주요국 중앙은행 등과의 글로벌 정책 논의에 적극적으로 참여하였다.

또한 국제컨퍼런스 및 세미나 개최를 통해 다양한 분야를 주제로 한 연구 결과를 관계기관 및 학계와 공유하고 상호 협력을 확대하였다.

특히 국제결제은행 산하 글로벌 금융시스템 위원회 의장직을 맡게 됨에 따라 글로벌 금융경제 현안에 대한 의제 선정 및 대응 정책 수립을 주도할 수 있는 계기가 마련되었다.


In [27]:
txt = ""

for doc in rectified_documents_step2:
    txt += doc.page_content

#txt = txt.replace("\n", " ")  # "\n" 제거

In [28]:
print(txt)
open('rectified_text_2023_3.txt', 'w', encoding='utf-8').write(txt)

주요 이슈

총재 서문 한국은행은 우리나라 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가안정과 금융안정을 도모함으로써 국민경제의 건전한 발전에 이바지하고 최선을 다하고 있다.

2023년은 급변하는 안팎의 여건에 대응하여 보다 나은 정책과 해법을 마련하는 데 분주했던 한 해였다.

인플레이션을 잡기 위한 주요국 중앙은행들의 가파른 금리 인상에 이어 미국 실리콘 밸리은행 사태, 이스라엘-하마스 전쟁 등으로 경계의 끈을 한시도 늦출 수 없는 긴장된 순간의 연속이었다.

이처럼 높아진 대외여건의 불확실성 속에서 한국은행은 최우선으로 물가안정을 도모하는 가운데 금융안정 측면의 리스크를 면밀히 점검하며 통화정책을 운영하였다.

기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금 인출 사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다.

한국은행은 경제상황에 대한 정확한 진단과 교묘한 정책 대응을 뒷받침하고자 조사 및 정책연구 업무를 적극적으로 수행하였다.

초저출산 및 초고령사회의 극단적인 구조 문제, 장기 구조적 관점에서 진단한 가계부채 현황, 지역 간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다.

또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각적으로 분석하는 한편 지급결제 환경 변화에도 선제적으로 대응하기 위해 중앙은행 디지털 화폐(CBDC) 활용성 테스트를 관계기관과 함께 추진하고 관련 연구를 이어나갔다.

한국은행은 국제사회에서 우리나라의 높아진 위상에 부합하는 역할을 수행하기 위해 국제기구, 주요국 중앙은행 등과의 글로벌 정책 논의에 적극적으로 참여하였다.

또한 국제컨퍼런스 및 세미나 개최를 통해 다양한 분야를 주제로 한 연구 결과를 관계기관 및 학계와 공유하고 상호 협력을 확대하였다.

특히 국제결제은행 산하 글로벌 금융시스템 위원회 의장직을 맡게 됨에 따라 글로벌 금융경제 현안에 대한 의제 선정 및 대응 정책 수립을 주도할 수 있는 계기가 마련되었다.


111185

In [29]:
import subprocess
subprocess.run(['say', '-v', 'Kyoko', 'プロセスが完了しました。'])

CompletedProcess(args=['say', '-v', 'Kyoko', 'プロセスが完了しました。'], returncode=0)